In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Library

In [ ]:
import PIL.Image as Image
import cv2
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd

%matplotlib inline
np.random.seed(42)

# Data

In [ ]:
df_train = pd.read_csv('../input/global-wheat-detection/train.csv')
df_train_img = '../input/global-wheat-detection/train'
df_test_img = '../input/global-wheat-detection/test'
submission = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [ ]:
df_train.head()

# Data Visualization

In [ ]:
sample_img = '../input/global-wheat-detection/train/005b0d8bb.jpg'
from IPython.display import Image, display
display(Image(sample_img,width=300, height=300))

In [ ]:
#helper fuction to show images
def show_images(image_ids):
    
    col = 5
    row = min(len(image_ids) // col, 5)
    
    fig, ax = plt.subplots(row, col, figsize=(16, 8))
    ax = ax.flatten()

    for i, image_id in enumerate(image_ids):
        image = cv2.imread('../input/global-wheat-detection/train' + '/{}.jpg'.format(image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ax[i].set_axis_off()
        ax[i].imshow(image)

In [ ]:
show_images(df_train.sample(n=15)['image_id'].values)

# Implementing YOLO v5 model

YOLO v5 requires the dataset to be in the darknet format. Here’s an outline of what it looks like:

* One txt with labels file per image
* One row per object
* Each row contains: class_index bbox_x_center bbox_y_center bbox_width bbox_height
* Box coordinates must be normalized between 0 and 1

We need two configuration files. One for the dataset and one for the model we’re going to use. I have included those file in my input as yolo-data file


In [ ]:
#cloning yolov5 model
!git clone https://github.com/ultralytics/yolov5

#cloning NVIDIA/apex to speed up the process
!git clone https://github.com/NVIDIA/apex.git

In [ ]:
!mv yolov5/* ./

In [ ]:
!pip install -r requirements.txt

In [ ]:
#Creating class_index bbox_x_center bbox_y_center bbox_width bbox_height
bboxs = np.stack(df_train['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df_train[column] = bboxs[:,i]
df_train.drop(columns=['bbox'], inplace=True)
df_train['x_center'] = df_train['x'] + df_train['w']/2
df_train['y_center'] = df_train['y'] + df_train['h']/2
df_train['classes'] = 0
from tqdm.auto import tqdm
import shutil as sh
df_train = df_train[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

In [ ]:
#Extracting index from train data
index = list(set(df_train.image_id))

In [ ]:
#creating dataset by divind the train data into train and val
source = 'train'
if True:
    for fold in [2]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df_train.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [ ]:
#Fit the Model
!python train.py --img 1024 --batch 2 --epochs 1 \
  --data ../input/yolo-data/wheat.yaml --cfg ../input/yolo-data/yolov5x.yaml --weights yolov5x.pt \
  --name yolov5x_fold2 --cache

# Prediction

In [ ]:
#taking test images as inference images
!find convertor/fold2/images/val2017 -maxdepth 1 -type f | head -50 | xargs cp -t "./inference/images/"

In [ ]:
#yolov5 in inference images
!python detect.py --weights weights/last_yolov5x_fold2.pt \
  --img 1024 --conf 0.4 --source ./inference/images/

# Visualize Result

In [ ]:
sample_img = 'inference/output/78edb3987.jpg'
from IPython.display import Image, display
display(Image(sample_img,width=500, height=500))

# If you like my kernel...Please upvote...